In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException       

#import helper libraries
import time
import urllib.request
import os
import requests
from PIL import Image

class GoogleImageScraper():
    def __init__(self,webdriver_path,image_path, search_key="cat",number_of_images=1,headless=False,min_resolution=(0,0),max_resolution=(1920,1080)):
        #check parameter types
        if (type(number_of_images)!=int):
            print("GoogleImageScraper Error: Number of images must be integer value.")
            return
        if not os.path.exists(image_path):
            print("GoogleImageScraper Notification: Image path not found. Creating a new folder.")
            os.makedirs(image_path)
        self.search_key = search_key
        self.number_of_images = number_of_images
        self.webdriver_path = webdriver_path
        self.image_path = image_path
        self.url = "https://www.google.com/search?q=%s&source=lnms&tbm=isch&sa=X&ved=2ahUKEwie44_AnqLpAhUhBWMBHUFGD90Q_AUoAXoECBUQAw&biw=1920&bih=947"%(search_key)
        self.headless=headless
        self.min_resolution = min_resolution
        self.max_resolution = max_resolution
        self.saved_extension = "jpg"
        self.valid_extensions = ["jpg","png","jpeg"]
        
    def find_image_urls(self):
        """
            This function search and return a list of image urls based on the search key.
            Example:
                google_image_scraper = GoogleImageScraper("webdriver_path","image_path","search_key",number_of_photos)
                image_urls = google_image_scraper.find_image_urls()
                
        """
        print("[+] Scraping for image link... Please wait.")
        image_urls=[]
        count = 0
        missed_count = 0
        options = Options()
        if(self.headless):
            options.add_argument('--headless')
        try:
            driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)
            driver.set_window_size(1400,1050)
            driver.get(self.url)
            time.sleep(5)
        except:
            print("[-] Please update the chromedriver.exe in the webdriver folder according to your chrome version:https://chromedriver.chromium.org/downloads")

        for indx in range (1,self.number_of_images+1):
            try:
                #find and click image
                imgurl = driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[%s]/a[1]/div[1]/img'%(str(indx)))
                imgurl.click()
                missed_count = 0 
            except Exception:
                #print("[-] Unable to click this photo.")
                missed_count = missed_count + 1
                if (missed_count>10):
                    print("[+] No more photos.")
                    break
                else:
                    continue
                 
            try:
                #select image from the popup
                time.sleep(1)
                class_names = ["n3VNCb"]
                images = [driver.find_elements_by_class_name(class_name) for class_name in class_names if len(driver.find_elements_by_class_name(class_name)) != 0 ][0]
                for image in images:
                    #only download images that starts with http
                    if(image.get_attribute("src")[:4].lower() in ["http"]):
                        print("%d. %s"%(count,image.get_attribute("src")))
                        image_urls.append(image.get_attribute("src"))
                        count +=1
                        break
            except Exception:
                print("[-] Unable to get link")   
                
            try:
                #scroll page to load next image
                driver.execute_script("window.scrollTo(0, "+str(indx*100)+");")
                element = driver.find_element_by_class_name("mye4qd")
                element.click()
                print("[+] Loading more photos")
                time.sleep(5)
            except Exception:  
                time.sleep(1)
         
        
        driver.close()
        print("[+] Google search ended")
        return image_urls

    def save_images(self,image_urls):
        #save images into file directory
        """
            This function takes in an array of image urls and save it into the prescribed image path/directory.
            Example:
                google_image_scraper = GoogleImageScraper("webdriver_path","image_path","search_key",number_of_photos)
                image_urls=["https://example_1.jpg","https://example_2.jpg"]
                google_image_scraper.save_images(image_urls)
                
        """
        print("[+] Saving Image... Please wait.")
        for indx,image_url in enumerate(image_urls):
            try:
                filename = "%s%s.%s"%(self.search_key,str(indx),self.saved_extension)
                image_path = os.path.join(self.image_path, filename)
                print("%d .Image saved at: %s"%(indx,image_path))
                image = requests.get(image_url)
                if image.status_code == 200:
                    with open(image_path, 'wb') as f:
                        f.write(image.content)
                        f.close()
                        image_from_web = Image.open(image_path)
                        image_resolution = image_from_web.size
                        if image_resolution != None:
                            if image_resolution[0]<self.min_resolution[0] or image_resolution[1]<self.min_resolution[1] or image_resolution[0]>self.max_resolution[0] or image_resolution[1]>self.max_resolution[1]:
                                image_from_web.close()
                                #print("GoogleImageScraper Notification: %s did not meet resolution requirements."%(image_url))
                                os.remove(image_path)
                        image_from_web.close()
            except Exception as e:
                print("GoogleImageScraper Error: Failed to be downloaded.",e)
                pass
        print("[+] Download Completed. Please note that some photos is not downloaded as it is not in the right format (e.g. jpg, jpeg, png)")

In [ ]:


webdriver_path = os.getcwd()+"\\webdriver\\chromedriver.exe"
image_path = os.getcwd()+"\\photos\\apple"
#add new search key into array ["cat","t-shirt","apple","orange","pear","fish"]
search_keys= ["cat","t-shirt"]
number_of_images = 20
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

In [16]:
search_keys= ["artificial ripe banana"]
number_of_images = 1000
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

[+] Scraping for image link... Please wait.


<ipython-input-6-4594d2cea1f5>:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)


0. https://www.snopes.com/tachyon/2019/05/Screen-Shot-2019-05-28-at-4.21.02-PM.png
1. https://qph.fs.quoracdn.net/main-qimg-9133ed4e387d9d0537efd9297f2c9556
2. https://www.ketchupmoms.com/uploads/blog/content/1508298265Watch%20Out.png
3. https://storage.ning.com/topology/rest/1.0/file/get/7364383652?profile=original
4. https://qph.fs.quoracdn.net/main-qimg-0e9d1361eb514ea87ca0ef0853734d30
5. https://www.researchgate.net/profile/Cyrus_Liu2/post/How_can_one_remove_the_calcium_carbide_content_in_banana/attachment/5ec1fcd9ead4db0001563a28/AS%3A892380421058562%401589771480977/download/IMG-20200517-WA0002.jpg
6. http://prachodayat.in/wp-content/uploads/2016/08/AgroFresh_Traditionell_gereifte_und_RipeLock-Bananen_Fancybox.jpg
7. https://image.slidesharecdn.com/seminar-161117074056/95/artificial-ripening-of-fruits-42-638.jpg?cb=1560759570
8. https://i.pinimg.com/originals/61/51/55/6151552513a72b7e3b68b0b32394ab61.jpg
9. https://www.snopes.com/tachyon/2019/05/banana-test.png
10. https://cms.pro

82. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ37PXagvIHz4XIzeH_ycZSnlgETowxb_9v3Q&usqp=CAU
83. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRJkVOiIAwF2ojutgutU-Hrj1BzAJyTP-S_Mg&usqp=CAU
84. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSiMetyhfx_4BAz2gSi7hku-2PDqZDFYBPaQA&usqp=CAU
85. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQKXaB8HuiSXho307RU_SWaLKTz507d3w4Tdw&usqp=CAU
86. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQvAbIJFx-gTKTmnbGaY0Xk0j5IjemWyqYX5A&usqp=CAU
87. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHqnVzURIyrrtjxN2a0zSCqg2P1MY3-XdcCA&usqp=CAU
88. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT8FxlRe4RJNtF13aV1LP7D4kphPQeVB7Kulg&usqp=CAU
89. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2gxZEXE6yeLC_FFoPiT_FjcaJzPFIY3r-_A&usqp=CAU
90. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSOcXhPgtIqUq3KnwyJNnXZ2q5sXzp0TUg7DQ&usqp=CAU
91. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS5Eob4

161. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT66sq69CDRaniIwGsmMfdJYKzg9pKtpNC_Qw&usqp=CAU
162. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqDQI4d153jmdin-3kkVq2APirBOM_MxRmag&usqp=CAU
163. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSsyB5_PUVDBzDtHyiGxXsBoruBnccuJFuGFg&usqp=CAU
164. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSKxClAEwRhLuApQwULirAnaL06-UqEIpmfXA&usqp=CAU
165. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2wHuP_fzCh-QHTLajrtuWBWivq-vzPLwbLA&usqp=CAU
166. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSXK0aEqqFq6Gh0iJTNw1k2hLFmJ-z_a3wvdg&usqp=CAU
167. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTG6ZgGT5I1_rWYn5GZrPpW4cB8CCk3betpbg&usqp=CAU
168. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTRrVwfHUkInnUcWLW-wcgkaiQKikkWZxhYNw&usqp=CAU
169. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRJ5_ahqtfqR7BTj0PLrkhiMrCc2qQivkfmVw&usqp=CAU
170. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

240. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqYkGTHKW1NQjvbQmucxvcI_Y8ntpUWOLV7A&usqp=CAU
241. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3JSnJ-bavxBR18HS8jgIOGim1bByt1xKfcg&usqp=CAU
242. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcStgKLDgL5WYXH6DCNJZU-O12WvdnDqlBlgdQ&usqp=CAU
243. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2sr0eOJOqqBZT_7ooEdMei4wJSFnNlzcapg&usqp=CAU
244. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSrmS97aSfYgKYEqhj31wqspIKp1I8fc-W_Vg&usqp=CAU
245. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQW5me07O2lMysOMPW2o-uMtO-6b2PVv36hHA&usqp=CAU
246. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSP6H89fBBdmbbJb2ourzksiTcMwNtncVNR4g&usqp=CAU
247. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRK-ICjkwzKfKrm7HTcHJDpQ3QRLqz863S5YQ&usqp=CAU
248. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSRugXtTNGmrhKm0C9J8Hu8Uf42mol6R6cPMQ&usqp=CAU
249. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

319. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYAvhrTF7XWWATZcVTUotlddkCj8ymXxI63Q&usqp=CAU
320. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxMoGbZJoMz95NzaaR9xIH8XTjfAPcC0qMKg&usqp=CAU
321. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRjxcYaomljNFlISvkv6ykfNlqpJr8VraFFmQ&usqp=CAU
322. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSs5ewBwfMwhL3epiwWaaFO0D2nhqFdAZ5Tew&usqp=CAU
323. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRpAsISX7ITjmc5dPiyO2lfG8fs20rEyNRUvA&usqp=CAU
324. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQg--ZKBe672rrb6PAsxIai296PALT2JjQ9ng&usqp=CAU
[+] No more photos.
[+] Google search ended
[+] Saving Image... Please wait.
0 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana0.jpg
1 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana1.jpg
2 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana2.jpg
3 .Image saved at: C:\Users\soora\photos\ban

89 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana89.jpg
90 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana90.jpg
91 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana91.jpg
92 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana92.jpg
93 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana93.jpg
94 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana94.jpg
95 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana95.jpg
96 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana96.jpg
97 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana97.jpg
98 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana98.jpg
99 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana99.jpg
100 .Image saved at: C:\Users\soora\photos\banana-grade2\artifici

184 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana184.jpg
185 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana185.jpg
186 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana186.jpg
187 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana187.jpg
188 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana188.jpg
189 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana189.jpg
190 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana190.jpg
191 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana191.jpg
192 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana192.jpg
193 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana193.jpg
194 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana194.jpg
195 .Image saved at: C:\Users\soora\photos\

279 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana279.jpg
280 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana280.jpg
281 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana281.jpg
282 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana282.jpg
283 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana283.jpg
284 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana284.jpg
285 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana285.jpg
286 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana286.jpg
287 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana287.jpg
288 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana288.jpg
289 .Image saved at: C:\Users\soora\photos\banana-grade2\artificial ripe banana289.jpg
290 .Image saved at: C:\Users\soora\photos\

In [7]:

import os

In [14]:
webdriver_path = os.getcwd()+"./chromedriver.exe"
image_path = os.getcwd()+"\\photos\\banana-grade2\\"

In [17]:
webdriver_path = os.getcwd()+"./chromedriver.exe"
image_path = os.getcwd()+"\\photos\\banana-grade3\\"

In [18]:
search_keys= ["rotten banana"]
number_of_images = 1000
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

GoogleImageScraper Notification: Image path not found. Creating a new folder.
[+] Scraping for image link... Please wait.


<ipython-input-6-4594d2cea1f5>:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)


0. https://image.shutterstock.com/image-photo/banana-bananas-isolated-on-white-260nw-594559439.jpg
1. https://i.pinimg.com/originals/8f/b2/36/8fb2366fdfece7d4df3098ceae751113.jpg
2. https://image.shutterstock.com/image-photo/rotten-banana-isolated-on-white-260nw-635638076.jpg
3. https://c8.alamy.com/comp/TT5DXN/rotten-banana-isolated-on-white-TT5DXN.jpg
4. https://previews.123rf.com/images/fotofritz/fotofritz1606/fotofritz160600034/59856909-rotten-banana-still-life.jpg
5. https://static.turbosquid.com/Preview/2019/06/29__11_49_06/banana_rotten_01.pngB14D454F-91E6-4649-84A4-2D9DF32C904ELarge.jpg
6. https://c8.alamy.com/comp/CWWNKB/rotten-banana-isolated-on-a-white-background-CWWNKB.jpg
7. https://images.unsplash.com/photo-1531171630952-6a86c2276cf7?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1000&q=80
8. https://st2.depositphotos.com/5089271/9609/i/950/depositphotos_96091810-stock-photo-rotten-banana-isolated-on-white.jpg
9. https://comps.cans

79. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT7CdwVwWBBFZ8Cg2rOQxI7ilkyHK4zs32W5g&usqp=CAU
80. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQNmPou6tlTVjUZ5OvACqovvvIXyVTy8_Ndhg&usqp=CAU
81. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRaFGrPd5fdW6N7HuI9iI5eo7LCoWIwPltDEg&usqp=CAU
82. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRZtJ8i8z9J3ui6-69xdbG_Hn46hv15MCKU5Q&usqp=CAU
83. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSpm_FJbYEObq9dUq0mzy6QAheY__nhfZHAiA&usqp=CAU
84. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfWHLcVE-XDgmSNgqShRePLYID7TwEQ4m4TQ&usqp=CAU
85. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTWX1jTAe741wcXt0bbf4APsQZUFAfER-F8WA&usqp=CAU
86. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRKV2qZrq87HKkpxtGD7W5M96v3qckmWJKIEA&usqp=CAU
87. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSgwySz3WMbvXagwIdulh2Rwd9Y0IvXctLwtw&usqp=CAU
88. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQkIVvh

158. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT3fim9Kc6y9aCJtV7tXYGdlKM0khL0sv7Gzw&usqp=CAU
159. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQT6_GLySvPHd5KSLwlTl9tyJltntCuCIca5Q&usqp=CAU
160. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSsp3f0HL2dW8iu4obiMpIX5xQDvoOtoJDb-w&usqp=CAU
161. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRxpxYHUPS4OqEmuNocT66y3WCXYUFb7CZIYA&usqp=CAU
162. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRyWy_w33Jgm4Ru3tXZOwhdrBGlJXjPABEUrA&usqp=CAU
163. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSplpgrS-QqcdUOLqfiGGzw1vRwaxWJuMvt2Q&usqp=CAU
164. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRgQBaZWVQ_UvuNfRIgDEu4mpMZ7hTg2R-8dA&usqp=CAU
165. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSmSWGM088VoXscxqb7GeYpIfA1RHvBjuv-8w&usqp=CAU
166. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTP9q-s30oUoVCkQhfGKvUnhPx-HeJ2HV58xg&usqp=CAU
167. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

...

GoogleImageScraper Error: Failed to be downloaded. cannot identify image file 'C:\\Users\\soora\\photos\\banana-grade3\\rotten banana15.jpg'
16 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana16.jpg
17 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana17.jpg
18 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana18.jpg
19 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana19.jpg
20 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana20.jpg
21 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana21.jpg
22 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana22.jpg
23 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana23.jpg
24 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana24.jpg
25 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana25.jpg
26 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana26.jpg
27 .Image saved at: C:\

122 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana122.jpg
123 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana123.jpg
124 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana124.jpg
125 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana125.jpg
126 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana126.jpg
127 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana127.jpg
128 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana128.jpg
129 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana129.jpg
130 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana130.jpg
131 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana131.jpg
132 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana132.jpg
133 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana133.jpg
134 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten 

228 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana228.jpg
229 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana229.jpg
230 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana230.jpg
231 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana231.jpg
232 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana232.jpg
233 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana233.jpg
234 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana234.jpg
235 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana235.jpg
236 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana236.jpg
237 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana237.jpg
238 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana238.jpg
239 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana239.jpg
240 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten 

334 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana334.jpg
335 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana335.jpg
336 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana336.jpg
337 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana337.jpg
338 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana338.jpg
339 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana339.jpg
340 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana340.jpg
341 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana341.jpg
342 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana342.jpg
343 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana343.jpg
344 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana344.jpg
345 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana345.jpg
346 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten 

440 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana440.jpg
441 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana441.jpg
442 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana442.jpg
443 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana443.jpg
444 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana444.jpg
445 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana445.jpg
446 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana446.jpg
447 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana447.jpg
448 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana448.jpg
449 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana449.jpg
450 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana450.jpg
451 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten banana451.jpg
452 .Image saved at: C:\Users\soora\photos\banana-grade3\rotten 

In [19]:
webdriver_path = os.getcwd()+"./chromedriver.exe"
image_path = os.getcwd()+"\\photos\\tomato-grade1\\"

In [20]:
search_keys= ["natural tomato","natural ripe tomato"]
number_of_images = 1000
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

GoogleImageScraper Notification: Image path not found. Creating a new folder.
[+] Scraping for image link... Please wait.


<ipython-input-6-4594d2cea1f5>:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)


0. https://5.imimg.com/data5/UD/BQ/MY-55946777/natural-red-tomato-500x500.jpg
1. https://cdn.shopify.com/s/files/1/0104/1059/0266/products/tomato.jpg?v=1575434681
2. https://www.planetnatural.com/wp-content/uploads/2013/03/organic-tomato-crop.jpg
3. https://commonsensehome.com/wp-content/uploads/2020/05/grow-tomatoes-organically-20.jpg
4. https://i0.wp.com/images-prod.healthline.com/hlcmsresource/images/AN_images/tomatoes-1296x728-feature.jpg?w=1155&h=1528
5. https://5.imimg.com/data5/LR/UL/FT/SELLER-38449243/natural-tomato-500x500.jpg
6. https://i.dailymail.co.uk/i/pix/scaled/2013/02/20/article-2281876-182583B1000005DC-875_308x185.jpg
7. https://5.imimg.com/data5/ANDROID/Default/2020/8/XD/LW/II/100045764/product-jpeg-500x500.jpg
8. https://www.mamanatural.com/wp-content/uploads/28-Things-to-Do-With-Too-Many-Tomatoes-tips-by-Mama-Natural.jpg
9. https://st3.depositphotos.com/3195613/14878/i/1600/depositphotos_148786801-stock-photo-organic-tomato-plant-and-fruit.jpg
10. https://images-eu

80. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTiRrN9PVoYj02o4gUAsjwRdmo4kRfD65VWPw&usqp=CAU
81. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0MjpK2a6tk9YGThSA0puSa-LJjwJTYmzGQw&usqp=CAU
82. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQrQt0BBuWHHZIOYetZl76HJ1uiow2_ZyMb2g&usqp=CAU
83. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSBbL_fjt00aCaa1AixkFAdtciVslfUBjw8Dg&usqp=CAU
84. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTVk_yraIiyGcpWOzbfyYPPHjxhYnfcIYvB7w&usqp=CAU
85. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT6j1Vaqv8PYrZe2lO2XJvcHS7QdKwxeWS1Zw&usqp=CAU
86. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSIKhXcOrIFDnUukiEWQXsYLqodwaUHXUOhNg&usqp=CAU
87. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSbbUyfEANxZq-TVp7256TFJ9PFT6Sz57Ql-w&usqp=CAU
88. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSc-kTI8MZ7Z44t3jNxbhNnnyublZNuUJob8Q&usqp=CAU
89. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYjYn-

159. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS9XbGWaFOvaxfSU4P74C2Cm9FbwGtRxGlWIQ&usqp=CAU
160. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT_S0g7fNt4hm85wRXfE_Qt_tHjiT_PEz0mxQ&usqp=CAU
161. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSymWMOfmWoNjsKSS8ed6O0jVzxfph066_moA&usqp=CAU
162. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRLW0I8LcpI8U_C5YYJTTVrFOW3w_zD0GqD9w&usqp=CAU
163. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTqcPl3h2ygpWPGBjnDQRBX5BjEBE4IaaFjFg&usqp=CAU
164. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ74OO8zh8Y8rMQ_pQU_NA7UJprGUY_O2mu7g&usqp=CAU
165. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSg-X09x4INn4s-gPBNf21skB3J6Qr31ARf4g&usqp=CAU
166. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR45pGaPAoB9Wb_VdZP7xYbqkCA9IVOeqEdqQ&usqp=CAU
167. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQQfyqYm0FWfP_qr4r0ogHE9cuFuZq1glxblA&usqp=CAU
168. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

...

341 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato341.jpg
342 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato342.jpg
343 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato343.jpg
344 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato344.jpg
345 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato345.jpg
346 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato346.jpg
347 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato347.jpg
348 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato348.jpg
349 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato349.jpg
350 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato350.jpg
351 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato351.jpg
352 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato

439 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato439.jpg
440 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato440.jpg
441 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato441.jpg
442 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato442.jpg
443 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato443.jpg
444 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato444.jpg
445 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato445.jpg
446 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato446.jpg
447 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato447.jpg
448 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato448.jpg
449 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato449.jpg
450 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato

537 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato537.jpg
538 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato538.jpg
539 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato539.jpg
540 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato540.jpg
541 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato541.jpg
542 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato542.jpg
543 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato543.jpg
544 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato544.jpg
545 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato545.jpg
546 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato546.jpg
547 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato547.jpg
548 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato

635 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato635.jpg
636 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato636.jpg
637 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato637.jpg
638 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato638.jpg
639 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato639.jpg
640 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato640.jpg
641 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato641.jpg
642 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato642.jpg
643 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato643.jpg
644 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato644.jpg
645 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato645.jpg
646 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato

733 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato733.jpg
734 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato734.jpg
735 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato735.jpg
736 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato736.jpg
737 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato737.jpg
738 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato738.jpg
739 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato739.jpg
740 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato740.jpg
741 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato741.jpg
742 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato742.jpg
743 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato743.jpg
744 .Image saved at: C:\Users\soora\photos\tomato-grade1\natural ripe tomato

In [21]:
webdriver_path = os.getcwd()+"./chromedriver.exe"
image_path = os.getcwd()+"\\photos\\tomato-grade2\\"

In [22]:
search_keys= ["artificial tomato","artificial ripe tomato"]
number_of_images = 1000
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

GoogleImageScraper Notification: Image path not found. Creating a new folder.
[+] Scraping for image link... Please wait.


<ipython-input-6-4594d2cea1f5>:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)


0. https://images-na.ssl-images-amazon.com/images/I/61bfC6pBBVL._SL1100_.jpg
1. https://images-na.ssl-images-amazon.com/images/I/61IjVRXQm5L._SL1500_.jpg
2. https://5.imimg.com/data5/XQ/OO/MY-36648926/organic-tomato-500x500.jpg
3. https://images-na.ssl-images-amazon.com/images/I/51UKtopj1bL._SL1000_.jpg
4. https://img.joomcdn.net/4affe9b99e510e0a2a324b7b569b8ae0699712a8_original.jpeg
5. https://images-na.ssl-images-amazon.com/images/I/71NqxXA1bZL._AC_SL1500_.jpg
6. https://i.ebayimg.com/images/g/5lgAAOSw1AVcrj6z/s-l300.jpg
7. https://f1af951e8abcbc4c70b9-9997fa854afcb64e87870c0f4e867f1d.lmsin.net/1000004855732-1000004855731_02-750-1.jpg
8. https://www.artplants.eu/media/catalog/product/cache/58/image/e4d92e6aceaad517e7b5c12e0dc06587/4/3/43217-nr-1.jpg
9. https://5.imimg.com/data5/DT/QB/QP/SELLER-6463342/firmenich-artificial-tomato-flavor-500x500.jpg
10. https://i.pinimg.com/originals/d2/37/d3/d237d3ed601d616ec2232c9ac73c16a4.jpg
11. https://www.rootsimple.com/wp-content/uploads/2013/02

82. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT2FShUQxVrTt8OB7NrHUXhxqocPmZ4hw-n_g&usqp=CAU
83. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTb6hBBw_B6h-h-pVDwNIDMHYtpCZOnx4KUkA&usqp=CAU
84. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRS-VmdqRkuo2udIxPHGMnIOu4ZdEyO2BJPeg&usqp=CAU
85. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQmplyXSDwCYetfbCthjlULi20SOyfs1g3mIw&usqp=CAU
86. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQtXN6SJakhcsWJ0OrF9YYUqt7jQlM9uB6uQg&usqp=CAU
87. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcStsW73Nzk-Swb_GgYNrstVJawLwsIpTvwLXQ&usqp=CAU
88. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSLMZuDdDewf4w4saJECWOxon1ZPktH-8cOkA&usqp=CAU
89. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQTDzNJjcJEf23X7ewPs1FtPy7l_GOMJxKybg&usqp=CAU
90. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRWdw-p8_sVWLOUbMsBO9TnvgEe1-05b_3uJg&usqp=CAU
91. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSCW6Pi

161. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTj0yE-fwII9E_ec1xp5P88gnOOMu0y0Ua9mw&usqp=CAU
162. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQlIphNRNCBH9INOCf8qta4CZZE6w2g6Hs_Tw&usqp=CAU
163. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSovnhKAzocEX-EReIjl3NQ0478zmndFcVflw&usqp=CAU
164. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSxuVQ7gxzr05YcyLjfQvSLfjcRpRlAN6l0jw&usqp=CAU
165. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQVVh8kasyBEk0LMKB6DOacV9AOPNs5Z4VeyA&usqp=CAU
166. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjY-m1PjKNc3RJ_FjrXxWg-yTod8e_hxht9Q&usqp=CAU
167. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSALUzq5DwPqeELOFviMoIN9qFrAPp1g_qSIw&usqp=CAU
168. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTSohqaZv9FafheSsrVdI7_turgJuiH_OlFfA&usqp=CAU
169. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQG1Eamz2BhD7XWWYWUdFqljAPy7rg8C0Elqw&usqp=CAU
170. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

...

270 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato270.jpg
271 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato271.jpg
272 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato272.jpg
273 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato273.jpg
274 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato274.jpg
275 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato275.jpg
276 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato276.jpg
277 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato277.jpg
278 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato278.jpg
279 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato279.jpg
280 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato280.jpg
281 .Image saved at: C:\Users\soora\photos\

365 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato365.jpg
366 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato366.jpg
367 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato367.jpg
368 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato368.jpg
369 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato369.jpg
370 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato370.jpg
371 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato371.jpg
372 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato372.jpg
373 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato373.jpg
374 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato374.jpg
375 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato375.jpg
376 .Image saved at: C:\Users\soora\photos\

460 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato460.jpg
461 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato461.jpg
462 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato462.jpg
463 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato463.jpg
464 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato464.jpg
465 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato465.jpg
466 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato466.jpg
467 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato467.jpg
468 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato468.jpg
469 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato469.jpg
470 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato470.jpg
471 .Image saved at: C:\Users\soora\photos\

555 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato555.jpg
556 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato556.jpg
557 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato557.jpg
558 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato558.jpg
559 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato559.jpg
560 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato560.jpg
561 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato561.jpg
562 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato562.jpg
563 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato563.jpg
564 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato564.jpg
565 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato565.jpg
566 .Image saved at: C:\Users\soora\photos\

650 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato650.jpg
651 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato651.jpg
652 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato652.jpg
653 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato653.jpg
654 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato654.jpg
655 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato655.jpg
656 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato656.jpg
657 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato657.jpg
658 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato658.jpg
659 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato659.jpg
660 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe tomato660.jpg
661 .Image saved at: C:\Users\soora\photos\

In [23]:
webdriver_path = os.getcwd()+"./chromedriver.exe"
image_path = os.getcwd()+"\\photos\\tomato-grade2\\"

In [24]:
search_keys= ["rotten tomato","unripe ripe tomato"]
number_of_images = 1000
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

[+] Scraping for image link... Please wait.


<ipython-input-6-4594d2cea1f5>:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)


0. https://media.gq.com/photos/59303fb5fd48c3741bfc222b/16:9/w_2560%2Cc_limit/rotten-tomatoes-scores.jpg
1. https://cdn.vox-cdn.com/thumbor/6qN5d4_wJsWe_keBmdw0_Fohrkc=/0x2749:3476x4903/1200x800/filters:focal(2660x4094:3216x4650)/cdn.vox-cdn.com/uploads/chorus_image/image/63125011/3163796.jpg.0.jpg
2. https://qph.fs.quoracdn.net/main-qimg-000084c2e8fa207c841380534ea72272
3. https://s.studiobinder.com/wp-content/uploads/2020/03/Rotten-Tomatoes-Ratings-Tomatometer-Score-Graphic-StudioBinder.jpg
4. https://daily-dew.com/wp-content/uploads/2018/11/rotten-tomatoes.jpg
5. https://image.shutterstock.com/image-photo/fresh-rotten-tomatoes-isolated-on-260nw-428871130.jpg
6. https://media.istockphoto.com/photos/rotten-tomato-picture-id182841392?k=6&m=182841392&s=612x612&w=0&h=gTLTVzltM9rue3Ux-bF9Npr4oIJMz_rBwWQ7kPnAqQQ=
7. https://qtxasset.com/styles/breakpoint_sm_default_480px_w/s3/2017-02/Rotten_tomatoes.jpg?_ZY69pLdU2dm2lcW2VQZeTxVQZm6Gryd&itok=M28J2T_S
8. https://miro.medium.com/max/564/0*jGZ

78. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT-5pzcyng4ldQaIVMZZ4zODwyQIq-oPROz0w&usqp=CAU
79. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTirKv8tckO877yg_UnIhcylQMs2_VNcbgI2w&usqp=CAU
80. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSTLo_zlaZySCzq3hGr1KP2sc7rCnKGZ66qzA&usqp=CAU
81. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS8IgLfEsHCVI3VoWSpFSNEztWOy9JNZTJbJQ&usqp=CAU
82. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRMFEh12o77mG3pk6HIbACeOVbozkD6NsGh1Q&usqp=CAU
83. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR5sjhYSOVH2W8IYjG2ulOxgEPr2NiZIb2WmA&usqp=CAU
84. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQte6YMjriXeWkxjwN3aLOXkzhCzQh31aM0oQ&usqp=CAU
85. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSCnEnLK3_KkOvF4j9hT0tZexA83R6am_dyiw&usqp=CAU
86. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT8kbP_x9ImmZwWudjWWKLtZsh0P8BuroHFdA&usqp=CAU
87. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTppRUg

157. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSa-5GzX9HzSZ_PWIQxyDf6DUKViDyOkvOBvA&usqp=CAU
158. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRXOnrWtU61OxPIz-BAFKzhFk71CiztPon1fA&usqp=CAU
159. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNqiN5SDx9ZobV7LBOr01OXJJ-1fLlFSD6GA&usqp=CAU
160. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSI626eNdYKbqPQ8T7iYNZ6Tw-GgcfGQ_t-4g&usqp=CAU
161. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQf2a6mYUIZ620sHGE7M9yuGWWnPTRvC4a9iA&usqp=CAU
162. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRetOmUSPuAWrRtS1_hIuGSn5vUEnzmvNpnHg&usqp=CAU
163. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRsqzditatrxHuv-JaA_RAivfl0tsc6Z9iulg&usqp=CAU
164. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWLe4RmcbIYVRDG9nufDb4qb8eYa-_8tBB0g&usqp=CAU
165. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRDNxQHByl59TPinfzjRlbU1Uv02VIdQySSTg&usqp=CAU
166. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

...

429 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato429.jpg
430 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato430.jpg
431 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato431.jpg
432 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato432.jpg
433 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato433.jpg
434 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato434.jpg
435 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato435.jpg
436 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato436.jpg
437 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato437.jpg
438 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato438.jpg
439 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato439.jpg
440 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato440.jpg
441 

529 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato529.jpg
530 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato530.jpg
531 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato531.jpg
532 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato532.jpg
533 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato533.jpg
534 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato534.jpg
535 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato535.jpg
536 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato536.jpg
537 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato537.jpg
538 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato538.jpg
539 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato539.jpg
540 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato540.jpg
541 

629 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato629.jpg
630 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato630.jpg
631 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato631.jpg
632 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato632.jpg
633 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato633.jpg
634 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato634.jpg
635 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato635.jpg
636 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato636.jpg
637 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato637.jpg
638 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato638.jpg
639 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato639.jpg
640 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato640.jpg
641 

728 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato728.jpg
729 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato729.jpg
730 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato730.jpg
731 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato731.jpg
732 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato732.jpg
733 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato733.jpg
734 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato734.jpg
735 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato735.jpg
736 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato736.jpg
737 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato737.jpg
738 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato738.jpg
739 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato739.jpg
740 

827 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato827.jpg
828 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato828.jpg
829 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato829.jpg
830 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato830.jpg
831 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato831.jpg
832 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato832.jpg
833 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato833.jpg
834 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato834.jpg
835 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato835.jpg
836 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato836.jpg
837 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato837.jpg
838 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe ripe tomato838.jpg
839 

In [25]:
webdriver_path = os.getcwd()+"./chromedriver.exe"
image_path = os.getcwd()+"\\photos\\tomato-grade2\\"

In [26]:
search_keys= ["natural carrot","natural ripe carrot"]
number_of_images = 1000
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

[+] Scraping for image link... Please wait.


<ipython-input-6-4594d2cea1f5>:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)


0. https://img3.exportersindia.com/product_images/bc-full/2019/6/6306499/fresh-natural-carrot-1559729344-4938082.jpeg
1. https://images-na.ssl-images-amazon.com/images/I/51bXCxWbsTL.jpg
2. https://thumbor.thedailymeal.com/DX2H8x_JCWEx-YL_PuInl9IPEEA=/870x565/filters:focal(2029x1416:2030x1417)/https://www.thedailymeal.com/sites/default/files/story/2017/shutterstock_154480130.jpg
3. https://www.greenprophet.com/wp-content/uploads/2010/06/coulorful-carrot.jpg
4. https://lh3.googleusercontent.com/proxy/rBGUat9YmOAzme7Tvf6bXXPclqFUsiRmYsi-1xI434Oyb3QZg45x7kj4AvMbygHBOoVxloYxCeXafBTlkc2hIRDQa2jQZ-dZWA
5. https://culturedecanted.files.wordpress.com/2014/06/a-natural-carrot.jpg?w=428
6. https://sc01.alicdn.com/kf/UT8zONBXvpbXXagOFbXk/102867024/UT8zONBXvpbXXagOFbXk.jpg_.webp
7. https://www.treehugger.com/thmb/E3qzj6r7ow8vtpJETMalJPmv9_A=/3264x2448/filters:no_upscale():max_bytes(150000):strip_icc()/GettyImages-1068390546-b0a53446c8204e01b3e5ea1cc2559501.jpg
8. https://www.treehugger.com/thmb/Srd

77. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS98fydkNkCmmpNgBLMlfPheyl2mPcw6V0qSg&usqp=CAU
78. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSEcKBJZES08XEbvTpXGk0xtzNIbGpThdLssQ&usqp=CAU
79. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRJcLp6_F_f7SFv2OKdDKZ07NH07tu8exW8rQ&usqp=CAU
80. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQpn7pQT4Dd4sFQLYFtRD9Ph9-eKqIvLxOHw&usqp=CAU
81. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT2EQwKFk8YBUuaduiSL9bAofxISCjtaibXmQ&usqp=CAU
82. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSjLveyuBbBiNW3mlcaCIP3dUBu31RhXaM7zg&usqp=CAU
83. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRuGMjsVdljQ50sw-Npats06VkppeEHF3m2-A&usqp=CAU
84. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRFjW370zMxm5DRRNOx1-fEk9U6W2pNX-gN1A&usqp=CAU
85. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ6CJEQgSCA9XockrKChUidvi9iQUtWClFTHA&usqp=CAU
86. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2jLDm

156. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRQMdYsTBOK0BvnyDhG1RaW_Xn4yXW-wxX9Aw&usqp=CAU
157. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTHeX-GXWcTC4kr3KgQEBkqYiRWxfE_PcODWQ&usqp=CAU
158. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSqbSQNqYEfvrbjOUPd0zBeXsVoGqZULlsgRA&usqp=CAU
159. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS5sshQRJcJsAzNeLuYxfyk95Q2SkRc0s3Vgg&usqp=CAU
160. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRFsEiwZg5R8JbhQS0FyOi2_1YcdRMLAgO4PA&usqp=CAU
161. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRHyhNTwT0ogKhRM72gtlDdPF4-EEf9BInifA&usqp=CAU
162. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQqNS71AH2B24wZM3tMfn5YlJst77XZoCEv3Q&usqp=CAU
163. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREqksTy9iqMeC4_gM-RlLmdNsGbyjYTTMbCw&usqp=CAU
164. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSczbDDg1ZaH2yhROd-jlFKBhGR2TnE60ucoQ&usqp=CAU
165. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

...

306 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot306.jpg
307 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot307.jpg
308 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot308.jpg
309 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot309.jpg
310 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot310.jpg
311 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot311.jpg
312 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot312.jpg
313 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot313.jpg
314 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot314.jpg
315 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot315.jpg
316 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot316.jpg
317 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot

404 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot404.jpg
405 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot405.jpg
406 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot406.jpg
407 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot407.jpg
408 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot408.jpg
409 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot409.jpg
410 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot410.jpg
411 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot411.jpg
412 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot412.jpg
413 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot413.jpg
414 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot414.jpg
415 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot

502 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot502.jpg
503 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot503.jpg
504 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot504.jpg
505 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot505.jpg
506 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot506.jpg
507 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot507.jpg
508 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot508.jpg
509 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot509.jpg
510 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot510.jpg
511 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot511.jpg
512 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot512.jpg
513 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot

600 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot600.jpg
601 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot601.jpg
602 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot602.jpg
603 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot603.jpg
604 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot604.jpg
605 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot605.jpg
606 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot606.jpg
607 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot607.jpg
608 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot608.jpg
609 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot609.jpg
610 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot610.jpg
611 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot

698 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot698.jpg
699 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot699.jpg
700 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot700.jpg
701 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot701.jpg
702 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot702.jpg
703 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot703.jpg
704 .Image saved at: C:\Users\soora\photos\tomato-grade2\natural ripe carrot704.jpg
[+] Download Completed. Please note that some photos is not downloaded as it is not in the right format (e.g. jpg, jpeg, png)


In [27]:
webdriver_path = os.getcwd()+"./chromedriver.exe"
image_path = os.getcwd()+"\\photos\\tomato-grade2\\"

In [28]:
search_keys= ["artificial carrot","artificial ripe carrot"]
number_of_images = 1000
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

[+] Scraping for image link... Please wait.


<ipython-input-6-4594d2cea1f5>:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)


0. https://images-na.ssl-images-amazon.com/images/I/51dLRV7p4UL._SY355_.jpg
1. https://i.pinimg.com/originals/93/6e/11/936e111289bf8c1d30d09b2be0243f05.jpg
2. https://images-na.ssl-images-amazon.com/images/I/311bTz-NbxL._AC_.jpg
3. https://images-na.ssl-images-amazon.com/images/I/41D2sa9HDTL.jpg
4. https://images-na.ssl-images-amazon.com/images/I/61RTE5ktrwL._SL1050_.jpg
5. https://i.ebayimg.com/images/g/bcIAAOSwpKJgRviN/s-l640.jpg
6. https://images-na.ssl-images-amazon.com/images/I/611iYKfbokL._SL1008_.jpg
7. https://www.zimmermanmarketplace.com/prodimages/car2.jpg
8. https://img.joomcdn.net/6026ff95ff7167e7fa753bd76fabd60fc8304aa2_original.jpeg
9. https://ae01.alicdn.com/kf/HLB1cn.qbinrK1RjSsziq6xptpXaO/Simulation-Carrot-Fruits-Artificial-Lifelike-Fake-Vegetables-Model-Home-Improvement-Craft-Jewelry-Kitchen-Photo-Props-Decoration.jpg
10. https://4.imimg.com/data4/TW/SK/ANDROID-7825117/product-500x500.jpeg
11. https://images-na.ssl-images-amazon.com/images/I/51wU8kgkJLL._AC_SL1008_.jp

82. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFtKo6ScGkREEFaK8nVrcajqOpS1hKGLy4fw&usqp=CAU
83. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQdHGAU39rbeNYet3NessXjDmDNz2_k2JTedA&usqp=CAU
84. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTkMsJq9OZJgxf8KCc5BEIxXwTIsmZDREr8GA&usqp=CAU
85. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSZ9GkSGlbKvyYTv1lIuXyP2HDIO-Q2YWgHuA&usqp=CAU
86. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQu-IvjHMNPV2KrmfEJJ71omJaRIGGE7sjVZw&usqp=CAU
87. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTWbRQBfARV5bncUVg64sL_YmB0HkzyksWRcg&usqp=CAU
88. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSXnqPbgsB6BDIbDJ6kmokmoIiNgtbcZtc1Sw&usqp=CAU
89. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRK0kgSWcU7BjfWqJeSSHU8aSASwtizphScUA&usqp=CAU
90. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4aoW6qAn5bgWPBU4Z-Cv-DTiWCtam8fZ1zQ&usqp=CAU
91. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQEKYHe

161. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSyeuC3W91OnGdZDVbH1oVbDBF2sU-Ai2vG3A&usqp=CAU
162. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRZJkRpORxH3xeO5_fuaIDBFOCzhAR6oilRjw&usqp=CAU
163. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRawTLYvNeN-zMJ93xbwX9T9SC45Rrb7J3baQ&usqp=CAU
164. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQyox0l2sV_Z0Ko4c9tyhr_OqCJxEX7LS2JWA&usqp=CAU
165. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR8IHBmwfcytOEKYmxMBJTBjJMRpC41FII9Ng&usqp=CAU
166. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBVPEmooQIWmTM2bsbWpsOgBnwbUrSiYzh2w&usqp=CAU
167. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTw64cZF1vb1u7k6IQIr4DEsg29F-tcUK3_-Q&usqp=CAU
168. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPHSam-6tOXZX5kVuI4Fq0Zb-1SVkYYuqLOw&usqp=CAU
169. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRCCsmcmtbP8Tdb7vLkZBJwfr4FeQ7QLva4Ew&usqp=CAU
170. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

...

259 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot259.jpg
260 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot260.jpg
261 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot261.jpg
262 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot262.jpg
263 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot263.jpg
264 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot264.jpg
265 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot265.jpg
266 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot266.jpg
267 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot267.jpg
268 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot268.jpg
269 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot269.jpg
270 .Image saved at: C:\Users\soora\photos\

354 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot354.jpg
355 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot355.jpg
356 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot356.jpg
357 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot357.jpg
358 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot358.jpg
359 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot359.jpg
360 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot360.jpg
361 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot361.jpg
362 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot362.jpg
363 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot363.jpg
364 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot364.jpg
365 .Image saved at: C:\Users\soora\photos\

449 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot449.jpg
450 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot450.jpg
451 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot451.jpg
452 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot452.jpg
453 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot453.jpg
454 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot454.jpg
455 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot455.jpg
456 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot456.jpg
457 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot457.jpg
458 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot458.jpg
459 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot459.jpg
460 .Image saved at: C:\Users\soora\photos\

545 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot545.jpg
546 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot546.jpg
547 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot547.jpg
548 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot548.jpg
549 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot549.jpg
550 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot550.jpg
551 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot551.jpg
552 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot552.jpg
553 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot553.jpg
554 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot554.jpg
555 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot555.jpg
556 .Image saved at: C:\Users\soora\photos\

640 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot640.jpg
641 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot641.jpg
642 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot642.jpg
643 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot643.jpg
644 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot644.jpg
645 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot645.jpg
646 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot646.jpg
647 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot647.jpg
648 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot648.jpg
649 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot649.jpg
650 .Image saved at: C:\Users\soora\photos\tomato-grade2\artificial ripe carrot650.jpg
651 .Image saved at: C:\Users\soora\photos\

In [29]:
webdriver_path = os.getcwd()+"./chromedriver.exe"
image_path = os.getcwd()+"\\photos\\carrot-grade2\\"

In [30]:
search_keys= ["rotten carrot","unripe carrot"]
number_of_images = 1000
headless = False
#min_resolution = (width,height)
min_resolution=(0,0)
#max_resolution = (width,height)
max_resolution=(1920,1080)
for search_key in search_keys:
    image_scrapper = GoogleImageScraper(webdriver_path,image_path,search_key,number_of_images,headless,min_resolution,max_resolution)
    image_urls = image_scrapper.find_image_urls()
    image_scrapper.save_images(image_urls)

[+] Scraping for image link... Please wait.


<ipython-input-6-4594d2cea1f5>:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(self.webdriver_path, chrome_options=options)


0. https://image.shutterstock.com/image-photo/rotten-carrot-white-background-260nw-691760755.jpg
1. https://thumbs.dreamstime.com/b/rotten-carrots-spoiled-moldy-vegetable-waste-wasted-food-close-up-sweaty-mouldy-rotting-vegetables-135710964.jpg
2. https://image.shutterstock.com/image-photo/rotten-carrot-isolated-on-white-260nw-453959623.jpg
3. https://media.istockphoto.com/photos/old-and-fresh-picture-id96657424?k=6&m=96657424&s=612x612&w=0&h=6q5a_OraqYDvRdI69D2Fzh8SGiwGmILXuHFHO9E5TzY=
4. https://image.shutterstock.com/image-photo/vegetable-diseases-brown-rot-carrot-260nw-1281756478.jpg
5. https://thumbs.dreamstime.com/z/four-rotten-carrots-stacked-white-background-clipping-path-177319030.jpg
6. https://images.pond5.com/rotten-carrot-white-background-footage-071043062_prevstill.jpeg
7. https://thumbs.dreamstime.com/b/rotten-carrots-wasted-black-mouldy-vegetables-gone-off-foo-rotten-carrots-wasted-black-mouldy-vegetables-crumple-104445827.jpg
8. https://image.shutterstock.com/image-pho

79. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSi3UwrmJOgxksyHiN2CcojpSOxrN-8SzVicA&usqp=CAU
80. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSrA9r4VyTb-he8vCcahgghUKzi8vZieZcXkg&usqp=CAU
81. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2C1uqkjv52M21ZpUbGEPuM6N64QdmIXW0PQ&usqp=CAU
82. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvqaACXb3sGSxX9feod_x7o361Xt6kcDguAQ&usqp=CAU
83. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ4rNNKD4Qb5qwqdq0_Vt5F9hPGFiGbBPPcxg&usqp=CAU
84. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQk8NYv1RyUiUWMdbPjwazJ3dyHoRx_qUphIA&usqp=CAU
85. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQwNgegxtaEwBzrjZLB-I0M_LM4BUlANJWxg&usqp=CAU
86. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQKLNIPPOlahtScZcC9_3ZKCXRkLRZL0CQZpg&usqp=CAU
87. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4_Ix-CvbQjOK2HEmwWnms3gP4fCYsy47URQ&usqp=CAU
88. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRv9waD

158. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRmuucM8EgCyjFwLgJ2ncd_YdCw9jmy3PFI6Q&usqp=CAU
159. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQs7JyShCWMnJY2L2Y9EwpIul1KgYtw7WdTpw&usqp=CAU
160. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQD6-P6GjL9OR3RFaOCI5fK9y0lnWLuLeXkXg&usqp=CAU
161. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR0wWR0vDt3dMthBUEHhTi5EUglfS4jbUcJ-g&usqp=CAU
162. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR-3jZwtPA_DLvMB5rpkt4fMwEqWHWMIv3pOg&usqp=CAU
163. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT3nYyjnI__RXmr8AlT2c-DekvG2-IsKtRpUg&usqp=CAU
164. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS9piDGQRFT7sWlkMCkzSEhEuRcugaJ_3J6fw&usqp=CAU
165. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS43bk20ACNoGIIF6baYF_jDZG3k2ehaJBeOw&usqp=CAU
166. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQa2XzvTKFlv109g_39kq9CKioejCwBur07YQ&usqp=CAU
167. https://encrypted-tbn0.gstatic.com/images?q=tbn:AN

...

293 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot293.jpg
294 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot294.jpg
295 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot295.jpg
296 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot296.jpg
297 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot297.jpg
298 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot298.jpg
299 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot299.jpg
300 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot300.jpg
301 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot301.jpg
302 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot302.jpg
303 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot303.jpg
304 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot304.jpg
305 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe 

399 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot399.jpg
400 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot400.jpg
401 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot401.jpg
402 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot402.jpg
403 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot403.jpg
404 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot404.jpg
405 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot405.jpg
406 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot406.jpg
407 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot407.jpg
408 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot408.jpg
409 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot409.jpg
410 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot410.jpg
411 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe 

505 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot505.jpg
506 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot506.jpg
507 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot507.jpg
508 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot508.jpg
509 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot509.jpg
510 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot510.jpg
511 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot511.jpg
512 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot512.jpg
513 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot513.jpg
514 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot514.jpg
515 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot515.jpg
516 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot516.jpg
517 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe 

611 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot611.jpg
612 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot612.jpg
613 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot613.jpg
614 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot614.jpg
615 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot615.jpg
616 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot616.jpg
617 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot617.jpg
618 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot618.jpg
619 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot619.jpg
620 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot620.jpg
621 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot621.jpg
622 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot622.jpg
623 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe 

717 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot717.jpg
718 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot718.jpg
719 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot719.jpg
720 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot720.jpg
721 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot721.jpg
722 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot722.jpg
723 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot723.jpg
724 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot724.jpg
725 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot725.jpg
726 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot726.jpg
727 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot727.jpg
728 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe carrot728.jpg
729 .Image saved at: C:\Users\soora\photos\tomato-grade2\unripe 